In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 为广告定向训练一个多类别分类模型
<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/ads_targetting/training-multi-class-classification-model-for-ads-targeting-usecase.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 上运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/ads_targetting/training-multi-class-classification-model-for-ads-targeting-usecase.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/ads_targetting/training-multi-class-classification-model-for-ads-targeting-usecase.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

## 简介

本教程演示了如何构建一个用于广告定向投放的机器学习模型。广告定向投放是一种广告技术，根据客户的过去行为和偏好，选择或定制广告展示给客户。定向广告旨在根据人口统计数据、心理统计数据、行为数据以及通常通过从客户收集的数据学习得到的其他二阶活动，来接触特定客户。

*注意：如果您正在使用[Vertex AI Workbench 托管笔记本](https://cloud.google.com/vertex-ai/docs/workbench/managed/create-instance)实例，请使用 `TensorFlow 2 (Local)` 内核。在其他笔记本环境中可能无法运行本笔记本的某些组件。*

了解更多关于[Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction)和[Vertex AI Training](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何从BigQuery中收集数据，预处理数据，并在电子商务数据集上训练一个多类别分类模型。

本教程使用以下Google Cloud ML服务和资源：

- BigQuery

执行的步骤包括：

- 从BigQuery获取所需数据
- 预处理数据
- 训练一个TensorFlow（>=2.4）分类模型
- 评估训练模型的损失
- 使用执行器功能自动执行笔记本
- 将模型保存到Cloud Storage路径
- 清理创建的资源

数据集

本教程使用BigQuery中的`looker-private-demo.ecomm`数据集。该数据集包含有关各种广告活动的信息，包括点击广告并在看到广告后进行购买的用户的人口统计信息。对于本教程，从该数据集中选择了来自美国的前三个广告活动，并将购买的用户信息用于训练一个模型，广告活动作为类别。其目的是看广告和用户数据是否可以用来识别哪种广告活动最适合用户。

该数据集可以通过在BigQuery中固定`looker-private-demo`项目来访问。如果您使用的是Vertex AI Workbench托管的笔记本实例，可以通过JupyterLab用户界面而不是BigQuery用户界面执行此过程。Vertex AI Workbench托管的笔记本实例支持通过其BigQuery集成浏览BigQuery中的数据集和表。

<img src="images/Bigquery_UI_new.PNG"></img>

### 费用

本教程使用了谷歌云的计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)、[BigQuery 价格](https://cloud.google.com/bigquery/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

### 安装额外的软件包

In [ ]:
! pip3 install --upgrade --quiet pandas-gbq \
                                 'google-cloud-bigquery[bqstorage,pandas]' \
                                 tensorflow \
                                 scikit-learn \
                                 numpy \
                                 protobuf==3.20.3

仅限Colab使用：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 设置地区

**可选**: 更新 'REGION' 变量以指定您想要使用的地区。了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行现场教程，请使用共享的测试帐户或项目。为了避免用户之间在创建的资源上发生名称冲突，您需要为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud帐户

要验证您的Google Cloud帐户，请按照您Jupyter环境的说明进行操作：

**1. Vertex AI Workbench**
<br>您已经通过验证。

**2. 本地JupyterLab实例**
<br>取消注释并运行以下代码：

In [ ]:
# ! gcloud auth login

取消注释并运行以下代码：

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

4. 服务帐户或其他  
* 请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务帐户授予云存储权限。

### 创建一个云存储桶

创建一个存储桶，用于存储诸如数据集等中间生成的工件。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在的情况下，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

导入库并定义常量

In [ ]:
import os
import warnings

import pandas as pd
from google.cloud.bigquery import Client
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

warnings.filterwarnings("ignore")

## 教程

### 从BigQuery获取数据
如果您正在使用***Vertex AI Workbench托管的笔记本实例***，以下以"#@bigquery"开头的单元格将是一个SQL查询。如果您正在使用Vertex AI Workbench用户管理的笔记本实例或Colab，则将是一个Markdown单元格。

#@bigquery

与
  traindata AS (
  选择
    b.* 除了(ad_event_id,
      user_id),
    c.* 除了(id),
    d.* 除了(keyword_id,
      ad_id),
    a.amount,
    a.device_type,
    e.name
  从
    `looker-private-demo.ecomm.ad_events` a
  连接 (
    选择
      ad_event_id,
      user_id,
      state,
      os,
      browser
    从
      `looker-private-demo.ecomm.events`
    其中
      event_type="Purchase"
      并且 country="USA") b
  在
    a.id = b.ad_event_id
  连接 (
    选择
      id,
      gender,
      age
    从
      `looker-private-demo.ecomm.users`) c
  在
    b.user_id = c.id
  连接 (
    选择
      keyword_id,
      ad_id,
      cpc_bid_amount,
      bidding_strategy_type,
      quality_score,
      keyword_match_type
    从
      `looker-private-demo.ecomm.keywords`
    其中
      cpc_bid_amount <= 3000) d
  在
    a.keyword_id = d.keyword_id
  连接 (
    选择
      ad_id,
      name
    从
      `looker-private-demo.ecomm.ad_groups`) e
  在
    d.ad_id = e.ad_id )
选择
  *
从
  traindata

如果您正在使用Vertex AI Workbench管理的笔记本实例，一旦从BigQuery显示的结果显示在上述单元格中，请点击**执行查询并加载为DataFrame**按钮并执行生成的代码存根以将数据获取到当前笔记本作为数据框中。

*注意：默认情况下数据加载到一个“df”变量中，但如果需要，在执行单元格之前可以更改这个变量名。*

In [ ]:
client = Client(project=PROJECT_ID)

In [ ]:
query = """WITH traindata AS (
SELECT b.* except(ad_event_id, user_id), c.* except(id), d.* except(keyword_id, ad_id), a.amount, a.device_type, e.name
FROM `looker-private-demo.ecomm.ad_events` a
JOIN
(SELECT ad_event_id, user_id, state, os, browser from `looker-private-demo.ecomm.events` WHERE event_type="Purchase" AND country="USA") b
ON a.id = b.ad_event_id
JOIN
(SELECT id, gender, age FROM `looker-private-demo.ecomm.users`) c
ON b.user_id = c.id
JOIN
(SELECT keyword_id, ad_id, cpc_bid_amount, bidding_strategy_type, quality_score, keyword_match_type FROM `looker-private-demo.ecomm.keywords`
WHERE cpc_bid_amount <= 3000) d
ON a.keyword_id = d.keyword_id
JOIN
(SELECT ad_id, name FROM `looker-private-demo.ecomm.ad_groups`) e
ON d.ad_id = e.ad_id
)
SELECT * FROM traindata"""
job = client.query(query)
df = job.to_dataframe()

### 数据预处理
从电子商务数据中选择必要的列，并根据它们的类型（数值/分类）进行划分。

In [ ]:
target = "name"
categ_cols = [
    "state",
    "os",
    "browser",
    "gender",
    "bidding_strategy_type",
    "keyword_match_type",
    "device_type",
]
num_cols = ["age", "cpc_bid_amount", "quality_score", "amount"]

从当前数据集中，将选择前三个最佳的广告活动来针对用户。有关广告和购买物品的用户的所有相关信息已经存储在数据框中。

In [ ]:
df = df[df["name"].isin(["Tops & Tees", "Active", "Accessories"])]

编码目标变量。

In [ ]:
df["name"] = df["name"].map({"Tops & Tees": 0, "Active": 1, "Accessories": 2})

将分类变量进行独热编码
在进行独热编码后，会删除第一个水平列，以避免[虚拟变量陷阱](https://en.wikipedia.org/wiki/Dummy_variable_(statistics))的情况发生。这个过程被称为*虚拟编码*。

In [ ]:
def encode_cols(data, col):
    # Creating a dummy variable for the variable 'CategoryID' and dropping the first one.
    categ = pd.get_dummies(data[col], prefix=col, drop_first=True)
    # Adding the results to the master dataframe
    data = pd.concat([data, categ], axis=1)
    return data


# dummy-encode the categorical fields
for i in categ_cols:
    df = encode_cols(df, i)
    df.drop(columns=[i], inplace=True)

# check the data's shape
df.shape

把数据分成训练集和测试集

In [ ]:
X = df[[i for i in df.columns if i != target]].copy()
y = df[target].copy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=36
)
print(X_train.shape, X_test.shape)

### 缩放数据

In [ ]:
scaler = StandardScaler()
X_train.loc[:, num_cols] = scaler.fit_transform(X_train[num_cols])
X_test.loc[:, num_cols] = scaler.transform(X_test[num_cols])

使用TensorFlow训练模型
将目标列转换为一个分类编码列（一热编码）。

In [ ]:
y_train_categ = to_categorical(y_train)
y_test_categ = to_categorical(y_test)

定义模型训练的超参数

* 注意：如果这些参数已经通过执行器功能作为输入参数提供，请在以下单元格中注释或删除。*

In [ ]:
optimizer = "sgd"
num_hidden_layers = 3
num_neurons = [64, 128, 256]
activ_func = ["relu", "relu", "relu"]

定义架构并编译模型。

In [ ]:
model = Sequential()
# construct the neural network as per the defined parameters
for i in range(num_hidden_layers):
    if i == 0:
        # add the input layer
        model.add(
            Dense(
                num_neurons[i],
                activation=activ_func[i],
                input_shape=(X_train.shape[1],),
            )
        )
    else:
        # add the hidden layers
        model.add(Dense(num_neurons[i], activation=activ_func[i]))

# add the output layer
model.add(Dense(3, activation="softmax"))
# compile the model
model.compile(loss="categorical_crossentropy", optimizer=optimizer)
model.summary()

训练模型

In [ ]:
import numpy as np

X_train = np.asarray(X_train, dtype=np.float32)

history = model.fit(X_train, y_train_categ, epochs=50, verbose=1)

### 在测试数据上评估模型。

In [ ]:
X_test = np.asarray(X_test, dtype=np.float32)

test_results = model.evaluate(X_test, y_test_categ, verbose=1)
print(f"Test results - Loss: {test_results}")

请注意：executor 功能仅在 Vertex AI Workbench 管理的笔记本中可用。

### 使用执行器在Vertex AI Workbench托管的笔记本实例中自动执行笔记本

如果您使用Vertex AI Workbench托管的笔记本实例，执行器可以帮助您从头到尾运行一个笔记本文件，选择环境、机器类型、输入参数和其他特性。设置执行后，笔记本将作为一个作业在Vertex AI自定义训练中执行。您可以从左侧菜单中的<b>笔记本执行器</b>面板监视作业。

执行器允许您选择环境和机器类型，同时自动运行类似于Vertex AI训练作业的运行，而无需切换到训练作业UI。除了默认复制现有内核的自定义容器外，还可以选择预构建环境如TensorFlow Enterprise、PyTorch和其他环境来运行笔记本。此外，可以通过从可用的机器类型列表中选择来指定所需的计算能力，包括GPU。

### 在 Vertex AI Workbench 托管的笔记本实例中安排执行器的运行

Vertex AI Workbench 托管的笔记本运行也可以使用执行器定期安排。要这样做，请选择 <b>基于计划的定期执行</b> 作为运行类型，而不是 <b>一次性执行</b>。在创建执行时，提供作业的频率和执行的时间。

<img src="images/executor_scheduled_runs2.png"></img>

### 参数化变量

如果您正在使用Vertex AI Workbench托管的笔记本实例，执行器可以让您使用不同的输入参数集运行笔记本。如有需要，笔记本中的常量可以被视为函数的参数，并在提交执行时，您可以将这些常量作为输入参数提供。

在模型训练步骤期间定义的超参数可以作为参数传递，同时提交执行。然而，在提交执行之前，应删除或将笔记本本身中定义的值注释掉。否则，输入参数将被笔记本中的值覆盖。

### 将模型保存到云存储路径

TensorFlow的`model.save()`方法支持将模型对象写入文件时使用云存储路径，也可以使用本地文件路径。需要确保用于运行此笔记本的服务账号具有对指定云存储路径的`写入`权限。

In [ ]:
GCS_PATH = BUCKET_URI + "/path-to-save/"
model.save(GCS_PATH)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

In [ ]:
# Delete the Cloud Storage bucket

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI